In [2]:
import org.apache.spark.ml.feature._
import org.apache.spark.ml.linalg._
import org.apache.spark.ml.stat._

In [43]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.spark.sql.SparkSession

In [4]:
val spark = SparkSession.builder()
      .master("local[1]")
      .appName("KMP_lab05_v1")
      .getOrCreate();
  println(spark)
  println("Spark Version : "+spark.version)

org.apache.spark.sql.SparkSession@6fd1328a
Spark Version : 2.4.7


spark = org.apache.spark.sql.SparkSession@6fd1328a


org.apache.spark.sql.SparkSession@6fd1328a

In [14]:
var train = spark.read
                 .option("header", true)
                 .option("inferSchema", "true")
                 .csv("/labs/slaba05/lab05_train.csv")

train = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [15]:
train.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = true)
 |-- APP_REGISTR_RGN_CODE: double (nullable = true)
 |-- PRC_ACCEPTS_A_POS: double (nullable = true)
 |-- PRC_ACCEPTS_A_TK: double (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: double (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CNT_ACCEPTS_TK: double (nullable = true)
 

In [28]:
train.select("_c0", "ID").sort("_c0").show(25)

Name: org.apache.spark.sql.AnalysisException
Message: cannot resolve '`_c0`' given input columns: [ID, AMOUNT_RUB_NAS_PRC, PRC_ACCEPTS_A_MTP, TURNOVER_DYNAMIC_IL_3M, LDEAL_DELINQ_PER_MAXYQZ, CNT_TRAN_SUP_TENDENCY1M, CR_PROD_CNT_CC, LDEAL_AMT_MONTH, REST_AVG_PAYM, PRC_ACCEPTS_A_TK, REST_DYNAMIC_PAYM_1M, TRANS_AMOUNT_TENDENCY3M, DEAL_YWZ_IR_MAX, MAX_PCLOSE_DATE, SUM_TRAN_ATM_TENDENCY3M, CR_PROD_CNT_CCFP, LDEAL_ACT_DAYS_PCT_TR, LDEAL_TENOR_MIN, TRANS_COUNT_SUP_PRC, LDEAL_YQZ_COM, APP_REGISTR_RGN_CODE, TURNOVER_PAYM, REST_DYNAMIC_SAVE_3M, CNT_ACCEPTS_TK, CNT_TRAN_CLO_TENDENCY3M, CNT_TRAN_AUT_TENDENCY3M, CNT_TRAN_AUT_TENDENCY1M, DEAL_YQZ_IR_MAX, TURNOVER_DYNAMIC_IL_1M, PRC_ACCEPTS_A_EMAIL_LINK, CR_PROD_CNT_VCU, DEAL_GRACE_DAYS_ACC_MAX, CR_PROD_CNT_IL, TRANS_COUNT_ATM_PRC, REST_DYNAMIC_CC_3M, DEAL_YWZ_IR_MIN, LDEAL_TENOR_MAX, REST_DYNAMIC_IL_1M, SUM_TRAN_MED_TENDENCY1M, CR_PROD_CNT_TOVR, REST_DYNAMIC_FDEP_3M, LDEAL_USED_AMT_AVG_YWZ, TRANS_COUNT_NAS_PRC, SUM_TRAN_AUT_TENDENCY3M, SUM_TRAN_ATM_

Очевидно что столбец "_с0" - это порядковый индекс в таблице

In [27]:
train = train.drop("_c0")

train = [ID: int, CR_PROD_CNT_IL: int ... 101 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 101 more fields]

In [16]:
var test = spark.read
                 .option("header", true)
                 .option("inferSchema", "true")
                 .csv("/labs/slaba05/lab05_test.csv")

test = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [17]:
test.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = true)
 |-- APP_REGISTR_RGN_CODE: double (nullable = true)
 |-- PRC_ACCEPTS_A_POS: double (nullable = true)
 |-- PRC_ACCEPTS_A_TK: double (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: double (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CNT_ACCEPTS_TK: double (nullable = true)
 

In [30]:
test = test.drop("_c0")

test = [ID: int, CR_PROD_CNT_IL: int ... 100 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 100 more fields]

In [9]:
val columns_cat = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", 
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", 
    "PACK"
)

columns_cat = List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)


List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)

In [31]:
train = train.drop(columns_cat:_*).na.fill(0)
test = test.drop(columns_cat:_*).na.fill(0)

train = [ID: int, CR_PROD_CNT_IL: int ... 101 more fields]
test = [ID: int, CR_PROD_CNT_IL: int ... 100 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 100 more fields]

In [32]:
val Array(trainData, valData) = train.randomSplit(Array(0.8, 0.2))

trainData = [ID: int, CR_PROD_CNT_IL: int ... 101 more fields]
valData = [ID: int, CR_PROD_CNT_IL: int ... 101 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 101 more fields]

In [33]:
trainData.columns

Array(ID, CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, CNT_TRAN_MED_TENDENCY1M, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, SUM_TRAN_CLO_TENDENCY1M, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CNT_TRAN_MED_TENDENCY3M, C...

# Моделька

In [19]:
import org.apache.spark.ml.classification.{GBTClassifier}
import org.apache.spark.ml.feature.{VectorAssembler}

In [34]:
val assembler = new VectorAssembler()
  .setInputCols(trainData.drop("ID", "TARGET").columns)
  .setOutputCol("features")

assembler = vecAssembler_b5b2d777f85f


vecAssembler_b5b2d777f85f

In [35]:
val trainDataF = assembler.transform(trainData).cache

trainDataF = [ID: int, CR_PROD_CNT_IL: int ... 102 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 102 more fields]

In [36]:
val gbtModel = new GBTClassifier()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    .setSeed(42)
    .fit(trainDataF)

gbtModel = GBTClassificationModel (uid=gbtc_a4f19f12a84f) with 20 trees


GBTClassificationModel (uid=gbtc_a4f19f12a84f) with 20 trees

# Test

In [37]:
val proba = gbtModel.transform(assembler.transform(test)).cache

probality = [ID: int, CR_PROD_CNT_IL: int ... 104 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 104 more fields]

In [52]:
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

vectorToArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


import org.apache.spark.mllib.linalg.{Vector=>OldVector}


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

In [55]:
val result = proba.withColumn("target", vectorToArrayUdf(col("probability")).getItem(1))
                      .withColumnRenamed("ID", "id")
                      .select("id", "target")

result = [id: int, target: double]


[id: int, target: double]

In [56]:
result.show(5)

+------+-------------------+
|    id|             target|
+------+-------------------+
|519130| 0.1546031201947734|
|234045|0.05032618584539683|
|401256|0.05007473625556291|
|551070|0.07114010925450454|
|367285|0.04955055689954857|
+------+-------------------+
only showing top 5 rows



In [57]:
result.write
      .option("header", true)
      .option("sep","\t")
      .mode("overwrite")
      .csv("file:////data/home/maxim.kondrat/lab05.csv")

In [58]:
spark.stop